In [ ]:
import sys
!{sys.executable} -m pip install imblearn
import numpy as np
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score as coh_kap
from sklearn.metrics import f1_score
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import sys

classes = []
data = []
names = []
fil=open("complex.csv","r")
x=0
for lin in fil.readlines():
    if(x==0):
        x=1
        header = lin.strip().split(",")[1:-2]
    else:
        flds = lin.strip().split(",")
        classes.append(flds[-1])
        data.append(flds[1:-4]+[flds[-3]])
        names.append(flds[0])
fil.close()


data = np.array(data).astype(np.float)
#classes = np.array(classes).astype(np.float)
data = MinMaxScaler().fit_transform(data)

X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size=0.2)

X_train, X_disc, y_train, y_disc = train_test_split(X_train, y_train, test_size=0.7)

data = X_train
classes = y_train

orig_data = X_test
orig_classes = y_test

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_sample(data, classes)

#print classes

data = X_resampled
classes = y_resampled
#data = RobustScaler().fit_transform(data)


# Random Forest
print ("Random Forest predictions")
feat = 80
while(feat < 140):
    trees = 100
    while(trees < 1000):
        clf = RandomForestClassifier(n_estimators=trees, max_features=feat)
        clf.fit(data, classes)
        rf_pred = clf.predict(orig_data)
        kap = coh_kap(orig_classes,rf_pred)
        f1 = f1_score(orig_classes,rf_pred,average='weighted')
        print ("RF\t"+str(feat)+"\t" +str(trees)+"\t"+str(kap)+"\t"+str(f1))
        trees = trees + 200
    feat = feat + 10



# Neural network
#print "Neural net predictions"
hidden_layer_sizes = [(2048,16),(1024,16),(512,16),(2048,64),(1024,64),(512,64),(2048,128),(1024,128),(512,128)]
alp = 0.1
while(alp < 5.0):
    for hid in hidden_layer_sizes:
        clf = MLPClassifier(solver='adam', alpha=alp, hidden_layer_sizes=hid,max_iter=1000)
        clf.fit(data, classes)
        cnn_pred = clf.predict(orig_data)
        kap = coh_kap(orig_classes,cnn_pred)
        f1 = f1_score(orig_classes,cnn_pred,average='weighted')
        print ("CNN\t"+str(alp)+"\t" +str(hid)+"\t"+str(kap)+"\t"+str(f1))
    alp=alp*2.0


# SVM params
#print "SVM predictions"
gam = 0.01
while(gam < 10.0):
    cval = 0.01
    while(cval < 10.0):
        clf = svm.SVC(kernel='rbf',gamma=gam,C=cval,max_iter=5000)
        clf.fit(data, classes)  
        svm_pred = clf.predict(orig_data)
        kap = coh_kap(orig_classes,svm_pred)
        f1 = f1_score(orig_classes,svm_pred,average='weighted')
        print ("SVM\t"+str(gam)+"\t" +str(cval)+"\t"+str(kap)+"\t"+str(f1))
        cval = cval * 2.0
    gam = gam * 2.0

